# Base de Dados Zoo
Identificar tipo de animal, dadas suas características

link da base de dados: https://archive.ics.uci.edu/ml/datasets/Zoo

### Proposta:
Combinar os modelos de classificação

In [1]:
# =====   Importação das Bibliotecas   ===== #
from pandas import read_csv, DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# Desabilita os warnings
import warnings
warnings.filterwarnings('ignore')

In [62]:
# Cria os modelos que mostraram ter melhor precisão
def create_models():
    forest_model = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state=0)
    forest_model.fit(previsores, classe)
    
    rna_model = MLPClassifier(max_iter=1000, tol=0.000001, solver='adam',
                           hidden_layer_sizes=(100), activation='relu',
                           batch_size=200, learning_rate_init=0.001)
    rna_model.fit(previsores, classe)

    NB_model = modelo = GaussianNB()
    NB_model.fit(previsores, classe)
    
    return forest_model, rna_model, NB_model

In [3]:
def analiza_registro(registro, nome_modelo):
    # Realiza as previsoes para cada um dos modelos
    # retorna as previsoes em um DataFrame
    dados = np.zeros((len(registro), len(nome_modelo)))
    previsoes = DataFrame(dados, columns=nome_modelo)

    for modelo, nome in zip(classificadores, nome_modelo):
        #print(modelo.predict(registro))
        previsoes[nome] = modelo.predict(registro)
    
    return previsoes

In [4]:
def combine_models(previsoes):
    # Define a classe do registro combinando as previsoes
    # utiliza o metodo de pluralidade (mais votos)
    # retorna as classificacoes definidas pelo metodo e
    #         os registros concatenanto as classificacoes com os atributos
    classificacoes = np.zeros(len(previsoes))
    classificacoes = classificacoes.reshape(-1,1)

    for i in range(len(previsoes)):
        lista = previsoes.iloc[i].values
        elemento, n_repeat = np.unique(lista, return_counts=True)
        aux = dict(zip(n_repeat, elemento))
        classificacoes[i] = aux[max(aux.keys())]
    
    registros = np.concatenate((novo_registro, classificacoes), axis=1)
    
    return registros, classificacoes

In [42]:
# Importação da base de dados Zoo
base = read_csv("zoo.csv")
atributos = list(base.columns[1:])

previsores = base.iloc[:, 1:17].values
classe = base.iloc[:, 17].values

In [63]:
# =====   Criação dos modelos a utilizar   ===== #
list_models = ['modelo_forest', 'modelo_rna', 'modelo_NB']

# Cria os modelos de classificacao e os salva em disco (opcional)
classificadores = create_models()
#save_models(classificadores, nomes=list_models)

In [64]:
# Avaliação dos modelos carregados
precisoes = DataFrame(columns=list_models)
for i in range(len(classificadores)):
    precisoes[list_models[i]] = Series(classificadores[i].score(previsores, classe))

precisoes

,modelo_forest,modelo_rna,modelo_NB
0,1.0,1.0,1.0


### Previsões da classificação de cada um dos modelos separados

In [65]:
# =====   Teste de Classificação de novo registro   ===== #
novo_registro = [['dog', 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 4, 1, 1, 1],
                ['eagle', 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0],
                ['butterfly', 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0],
                ['anteater', 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 4, 1, 0, 1],
                ['whale', 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1],
                ['komodo dragon', 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 4, 1, 0, 1]]
novo_registro = np.array(novo_registro)
att_animal = DataFrame(novo_registro[:, 0])
novo_registro = novo_registro[:, 1:].astype(int)

# Realização das previsoes
previsoes = analiza_registro(novo_registro, list_models)
previsoes

,modelo_forest,modelo_rna,modelo_NB
0,1,1,1
1,2,2,2
2,6,6,6
3,1,1,1
4,1,1,1
5,3,3,3


### Classificação final dos novo registros após combinação dos modelos

In [61]:
# =====   Combinação dos Classificadores   ===== #
registros, _ = combine_models(previsoes)
registros = DataFrame(registros, columns=atributos)
registros.insert(0, 'Animal', att_animal)

dict_classe = {1: 'mamífero', 2: 'ave', 3: 'reptil', 4: 'peixe', 5: 'anfibio', 6: 'inseto', 7: 'artropodes/moluscos'}

for i in dict_classe.keys():
    registros.loc[registros["type"] == i, "classe"] = dict_classe[i]

registros

,Animal,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type,classe
0,dog,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,4.0,1.0,1.0,1.0,1.0,mamífero
1,eagle,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,ave
2,butterfly,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,inseto
3,anteater,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,4.0,1.0,0.0,1.0,1.0,mamífero
4,whale,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,mamífero
5,komodo dragon,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,4.0,1.0,0.0,1.0,3.0,reptil
